# Import packages
---
---

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.pipeline import Pipeline
%config InlineBackend.figure_format = 'svg'
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
import warnings
warnings.filterwarnings('ignore')
import random

#from yellowbrick.regressor import prediction_error

# Get the Data
---

In [ ]:
#df=pd.read_csv('/Users/reema/Downloads/hotel_booking.csv')
df = pd.read_csv('hotel_booking.csv')
df

# Take a Look at the Data Structure
---

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
#Numeric
numerical = df.loc[:, (df.dtypes == 'int64') | (df.dtypes == 'float64')].columns.tolist()
numerical

In [ ]:
#Categorical
categorical = df.loc[:, (df.dtypes != 'int64') & (df.dtypes != 'float64')].columns.tolist()
categorical

In [ ]:
df[numerical].describe()

In [ ]:
df[categorical].describe()

In [ ]:
# Check classes for prediction
df['is_canceled'].value_counts()#(normalize=True)

In [ ]:
# Check value counts for catogrical columns
for col in categorical:
    print(f'''Value count column {col}:''')
    print('-' * 50)
    print(df[col].value_counts())
    print()

In [ ]:
# Check Duplicates
df.duplicated().sum()

In [ ]:
#df.hist(bins=60, figsize=(20,15));

# Split Data
---

In [ ]:
total_previous_cancellations = df['previous_cancellations'].unique() 
total_previous_cancellations

In [ ]:
#train/test split on randomly selected groups of previous_cancellations
np.random.seed(80)
total_previous_cancellations = df['previous_cancellations'].unique() 
test_previous_cancellations = np.random.choice(total_previous_cancellations, size=int(total_previous_cancellations.shape[0] * .20))

In [ ]:
df_train, df_test = df[~df['previous_cancellations'].isin(test_previous_cancellations)], df[df['previous_cancellations'].isin(test_previous_cancellations)]

In [ ]:
#train/val split on randomly selected groups of previous_cancellations
total_train_previous_cancellations = df_train['previous_cancellations'].unique() 
val_previous_cancellations = np.random.choice(total_train_previous_cancellations, size=int(total_train_previous_cancellations.shape[0] * .25))

In [ ]:
df_train, df_val = df_train[~df_train['previous_cancellations'].isin(val_previous_cancellations)], df_train[df_train['previous_cancellations'].isin(val_previous_cancellations)]

In [ ]:
print(f'Shape of train:', df_train.shape)
print(f'Shape of validation:', df_val.shape)
print(f'Shape of test:', df_test.shape)

# ------------------------------------------

In [ ]:
df_train['previous_cancellations'].unique()

In [ ]:
df_test['previous_cancellations'].unique()

In [ ]:
df_val['previous_cancellations'].unique()

# ------------------------------------------

In [ ]:
df_train['is_canceled'].value_counts()

In [ ]:
df_val['is_canceled'].value_counts()

In [ ]:
df_test['is_canceled'].value_counts()

## Data Pre-Processing

---

In [ ]:
df_train.isnull().sum()

In [ ]:
df_val.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
#Delete Irrelevant Features

# for train
df_train.drop(['name', 'email', 'phone-number', 'credit_card'], axis = 1, inplace=True)

# for val
df_val.drop(['name', 'email', 'phone-number', 'credit_card'], axis = 1, inplace=True)

# for teast
df_test.drop(['name', 'email', 'phone-number', 'credit_card'], axis = 1, inplace=True)


In [ ]:
#Remove rows that have no children data

# for train
df_train.dropna(subset=['children'], inplace=True)

# foe val
df_val.dropna(subset=['children'], inplace=True)


# for test
df_test.dropna(subset=['children'], inplace=True)


In [ ]:
#Change value company to 0 and 1

# for train
df_train['company'] = df_train['company'].apply(lambda x: 0 if pd.isnull(x) else 1)
df_train['company'].value_counts()

# for val
df_val['company'] = df_val['company'].apply(lambda x: 0 if pd.isnull(x) else 1)
df_val['company'].value_counts()


# for test
df_test['company'] = df_test['company'].apply(lambda x: 0 if pd.isnull(x) else 1)
df_test['company'].value_counts()

In [ ]:
#Change value agent to 0 and 1

# for train
df_train['agent'] = df_train['agent'].apply(lambda x: 0 if pd.isnull(x) else 1)
df_train['agent'].value_counts()

# for val
#Change value agent to 0 and 1
df_val['agent'] = df_val['agent'].apply(lambda x: 0 if pd.isnull(x) else 1)
df_val['agent'].value_counts()

# for test
#Change value agent to 0 and 1
df_test['agent'] = df_test['agent'].apply(lambda x: 0 if pd.isnull(x) else 1)
df_test['agent'].value_counts()

In [ ]:
#Fill NaN value in country with the value of mode

# for train
df_train['country'].fillna(df_train['country'].mode()[0], inplace = True)
df_train['country'].value_counts()

# for val
df_val['country'].fillna(df_val['country'].mode()[0], inplace = True)
df_val['country'].value_counts()

# for test
df_test['country'].fillna(df_test['country'].mode()[0], inplace = True)
df_test['country'].value_counts()

In [ ]:
df_train.isna().sum()

# Handel incorrect value
---

ADR

In [ ]:
df_train['adr'].describe()

In [ ]:
df_train[df_train.adr < 0]

In [ ]:

# for train
df_train.drop(df_train[df_train['adr'] < 0].index, inplace = True)

# for val
df_val.drop(df_val[df_val['adr'] < 0].index, inplace = True)

# for test
df_test.drop(df_test[df_test['adr'] < 0].index, inplace = True)

In [ ]:
df_train['adr'].describe()

Adults


In [ ]:
df_train['adults'].describe()

In [ ]:
# for train
df_train.drop(df_train[df_train['adults']<1].index, inplace= True)

# for val
df_val.drop(df_val[df_val['adults']<1].index, inplace= True)

# for test
df_test.drop(df_test[df_test['adults']<1].index, inplace= True)

In [ ]:
df_train['adults'].describe()

# Change Type

In [ ]:
# for train

# Changing children to integer type: 
df_train['children'] = df_train['children'].astype(int)

# Changing reservation status date to datetime type: 
df_train['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

# Changing agent to integer type: 
df_train['agent'] = df_train['agent'].astype(int)

# Changing company to integer type: 
df_train['company'] = df_train['company'].astype(int)

In [ ]:
# for train

# Changing children to integer type: 
df_val['children'] = df_val['children'].astype(int)

# Changing reservation status date to datetime type: 
df_val['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

# Changing agent to integer type: 
df_val['agent'] = df_val['agent'].astype(int)

# Changing company to integer type: 
df_val['company'] = df_val['company'].astype(int)

In [ ]:
# for train

# Changing children to integer type: 
df_test['children'] = df_test['children'].astype(int)

# Changing reservation status date to datetime type: 
df_test['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

# Changing agent to integer type: 
df_test['agent'] = df_test['agent'].astype(int)

# Changing company to integer type: 
df_test['company'] = df_test['company'].astype(int)

In [ ]:
df_train.info()

# Feature Engineering
---

Total Stays


In [ ]:
# for train
df_train['total_stays'] = df_train['stays_in_weekend_nights'] + df_train['stays_in_week_nights']

# for val
df_val['total_stays'] = df_val['stays_in_weekend_nights'] + df_val['stays_in_week_nights']

# for test
df_test['total_stays'] = df_test['stays_in_weekend_nights'] + df_test['stays_in_week_nights']

Total Guests

In [ ]:
# for train
df_train['total_guests'] = df_train['adults'] + df_train['children'] + df_train['babies']

# for val
df_val['total_guests'] = df_val['adults'] + df_val['children'] + df_val['babies']

# for test
df_test['total_guests'] = df_test['adults'] + df_test['children'] + df_test['babies']

kids

In [ ]:
# for train
df_train['kids'] = df_train['children'] + df_train['babies']

# for val
df_val['kids'] = df_val['children'] + df_val['babies']

# for test
df_test['kids'] = df_test['children'] + df_test['babies']

Guest location

In [ ]:
# for train
df_train['guest_location'] = df_train['country'].apply(lambda x: 'Local' if x == 'PRT' else 'International')
df_train['guest_location'].value_counts()

# for val
df_val['guest_location'] = df_val['country'].apply(lambda x: 'Local' if x == 'PRT' else 'International')
df_val['guest_location'].value_counts()

# for test
df_test['guest_location'] = df_test['country'].apply(lambda x: 'Local' if x == 'PRT' else 'International')
df_test['guest_location'].value_counts()

In [ ]:
# Delete 

# for train
df_train.drop(['stays_in_weekend_nights', 'stays_in_week_nights', 'children', 'babies'], axis = 1, inplace=True)

# for val
df_val.drop(['stays_in_weekend_nights', 'stays_in_week_nights', 'children', 'babies'], axis = 1, inplace=True)

# for test
df_test.drop(['stays_in_weekend_nights', 'stays_in_week_nights', 'children', 'babies'], axis = 1, inplace=True)


In [ ]:
# check Duplicates
df_train.duplicated().sum()


In [ ]:
df_val.duplicated().sum()

In [ ]:
df_test.duplicated().sum()

In [ ]:
# drop Duplicates
# for train
df_train.drop_duplicates(inplace=True)

# for val
df_val.drop_duplicates(inplace=True)

# for test
df_test.drop_duplicates(inplace=True)


In [ ]:
df_train.duplicated().sum()


In [ ]:
df_train['is_canceled'].value_counts()

# Get Dummies 

In [ ]:
df.dtypes

In [ ]:
df.nunique()

In [ ]:
df.reserved_room_type.value_counts()

In [ ]:
df.assigned_room_type.value_counts()

In [ ]:
df1=df1 = df_train.select_dtypes(exclude=['object']).copy()

In [ ]:
df_obj_train=df_train.loc[:,['hotel','arrival_date_month','market_segment','distribution_channel','assigned_room_type','deposit_type','customer_type','reservation_status','guest_location']]
           

In [ ]:
df_obj_val=df_val.loc[:,['hotel','arrival_date_month','market_segment','distribution_channel','assigned_room_type','deposit_type','customer_type','reservation_status','guest_location']]  

In [ ]:
df_obj_test=df_test.loc[:,['hotel','arrival_date_month','market_segment','distribution_channel','assigned_room_type','deposit_type','customer_type','reservation_status','guest_location']]  

In [ ]:
df_dum_tr = pd.get_dummies(df_obj_train, drop_first=True)

df_train1 = pd.concat([df1, df_dum_tr], axis=1)
df_train1.head(2)

In [ ]:
df_dum_val = pd.get_dummies(df_obj_val, drop_first=True)

df_val1 = pd.concat([df1, df_dum_val], axis=1)
df_val1.head(2)

In [ ]:
df_dum_test = pd.get_dummies(df_obj_test, drop_first=True)

df_test1 = pd.concat([df1, df_dum_test], axis=1)
df_test1.head(2)

In [ ]:
#sns.heatmap(df_train.corr(), cmap="seismic", annot=True, vmin=-1, vmax=1);


In [ ]:
df_train1.corr()

# EDA

In [ ]:
# plt.figure(figsize=(15, 25))
# for i in range(0, len(numerical)):
#     plt.subplot(10, int(len(numerical)/9), i+1)
#     sns.distplot(df_train[numerical[i]], color='gray')
#     plt.tight_layout()

In [ ]:
df_train.shape

In [ ]:
df_val.shape

In [ ]:
df_test.shape

In [ ]:
train_sample= df_train1[['is_canceled','lead_time','previous_cancellations','previous_bookings_not_canceled',
                           'assigned_room_type_K','assigned_room_type_L','deposit_type_Non Refund','deposit_type_Refundable',
                           'customer_type_Group','customer_type_Transient','customer_type_Transient-Party',
                           'reservation_status_Check-Out','reservation_status_No-Show','guest_location_Local',
                           'hotel_Resort Hotel','market_segment_Complementary','market_segment_Corporate',
                           'market_segment_Direct','market_segment_Groups','market_segment_Offline TA/TO',
                           'market_segment_Online TA','distribution_channel_Direct','distribution_channel_GDS',
                           'distribution_channel_TA/TO','distribution_channel_Undefined','assigned_room_type_B',
                           'assigned_room_type_C','assigned_room_type_D','assigned_room_type_E','assigned_room_type_F',
                           'assigned_room_type_G','assigned_room_type_H','assigned_room_type_I','assigned_room_type_K',
                           'assigned_room_type_L','deposit_type_Non Refund','deposit_type_Refundable','customer_type_Group',
                           'customer_type_Transient','customer_type_Transient-Party','reservation_status_Check-Out',
                           'reservation_status_No-Show'
                           ]]







val_sample = df_val1[['hotel_Resort Hotel','is_canceled','lead_time','previous_cancellations','distribution_channel_Direct',
                      'distribution_channel_TA/TO','market_segment_Complementary','market_segment_Corporate',
                      'market_segment_Direct',
                      'market_segment_Groups','market_segment_Offline TA/TO','market_segment_Online TA',
                      'deposit_type_Non Refund','previous_bookings_not_canceled','customer_type_Group','customer_type_Transient',
                      'customer_type_Transient-Party','assigned_room_type_B','assigned_room_type_C',
                      'assigned_room_type_D','assigned_room_type_E','assigned_room_type_F','assigned_room_type_G',
                      'assigned_room_type_H'
                     ]]
    
                          

test_sample = df_test1[['is_canceled','lead_time','previous_cancellations','previous_bookings_not_canceled',
                        'hotel_Resort Hotel','market_segment_Direct',
                      'market_segment_Groups','market_segment_Offline TA/TO','market_segment_Online TA',
                        'market_segment_Corporate','distribution_channel_Direct','distribution_channel_TA/TO',
                        'assigned_room_type_B','assigned_room_type_D','assigned_room_type_E','assigned_room_type_F',
                        'assigned_room_type_G','deposit_type_Non Refund','reservation_status_Check-Out',
                        'reservation_status_No-Show','customer_type_Transient-Party']]
    
    
   

In [ ]:
# Classes
plt.figure(figsize=(5, 6))
cancel_ax = sns.countplot(df_train['is_canceled'])

for p in cancel_ax.patches:
    cancel_ax.annotate(format(p.get_height(), '.0f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')

## Cancellation Rate

In [ ]:
labels = ['Confirmed Booking',  'Cancelled Booking']
myexplode = [0.2, 0]
cancel_fig, cancel_ax = plt.subplots(figsize=[10,6])
cancel_ax.pie(df_train['is_canceled'].value_counts(), autopct='%1.1f%%',
        shadow=True, startangle=90, explode = myexplode, textprops={'color':"black", 'fontsize':20}, labels=labels)
cancel_ax.axis('equal'); 

---
## What is the month in which there were the most cancellations, and what is the month in which there were the least cancellations?

In [ ]:
# plt.figure(figsize=(10, 10))
# sns.barplot(df_train['arrival_date_month'], df_clean['is_canceled'], color='steelblue')

# plt.xlabel('arrival_date_month', fontsize=12)
# plt.ylabel('is_canceled', fontsize=12)
# plt.title('Cancellation Rate By Months', fontsize=20, pad=30)
# plt.show()

# Select what year was the most reservation in?

In [ ]:
counts_year = df_train.arrival_date_year.value_counts().sort_values(ascending=False).reset_index()
counts_year.columns = ['year', 'count']
counts_year

In [ ]:
sns.barplot(x = 'year', y = 'count', data = counts_year , color = '#4682B4')
plt.xlabel('arrival date year')
plt.ylabel('counts')
plt.title('The Most reservation  year',fontsize =10,weight='bold',color='black')

# What is the cancellation rate for each hotel?

In [ ]:
Most_hotel_cancellation = df_train.groupby('hotel').is_canceled.sum()
Most_hotel_cancellation

In [ ]:
fig = plt.figure(figsize =(7,7))
c = ['#4682B4','darkorange'
plt.title('Cancellation rate per hotel', fontsize = 12, weight = 'bold')
plt.pie(Most_hotel_cancellation , labels=Most_hotel_cancellation.index,autopct="%0.1f%%", pctdistance=0.6 ,colors=c);

# Is the number of changes of the reservation affected by (market_segment)?

In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(y=df_train.booking_changes,x=df_train.market_segment
            ,ci=True,color='steelblue')
plt.ylabel('booking_changes')
plt.xlabel('market_segment');


# Is booking changes affected by customer type or by days in waiting list?

In [ ]:
#plt.bar(df_train.booking_changes,df_train.days_in_waiting_list);

In [ ]:
#plt.bar(df_train.booking_changes,df_train.customer_type);

---
## Scaler

In [ ]:
scaler = StandardScaler()

train_sample[train_sample.columns]=scaler.fit_transform(train_sample[train_sample.columns])
val_sample[val_sample.columns]=scaler.transform(val_sample[val_sample.columns])
test_sample[test_sample.columns]=scaler.transform(test_sample[test_sample.columns])

---

## KNN Model

In [ ]:
X_train=train_sample
y_train=df_train1['is_canceled']
X_val=val_sample
y_val=df_val1['is_canceled']
X_test=test_sample
y_test=df_test1['is_canceled']

In [ ]:

from sklearn.metrics import f1_score


knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
print('The accuracy for training : ',knn.score(X_train,y_train))
print('The accuracy for validation: ',knn.score(X_val,y_val))
print('f1_score for train: ',f1_score(knn.predict(X_train), y_train))
print('f1_score for validation: ',f1_score(knn.predict(X_val), y_val))